In [ ]:
!pip install -q streamlit pyngrok transformers accelerate bitsandbytes langchain huggingface_hub gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [ ]:
!pip freeze > requirements.txt

In [ ]:
from google.colab import files
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%writefile phi3_helper.py
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load model and tokenizer (GPU + bf16)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-3-mini-4k-instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto"  # automatically uses GPU
)

def generate_article(topic):
    prompt = f"""Write a well-researched, clear, and engaging article about the following topic:\n\nTopic: {topic}\n\nArticle:"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=500, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
%%writefile main.py
import streamlit as st
from phi3_helper import generate_article

st.set_page_config(page_title="Phi-3 Mini Article Generator", page_icon="📝")
st.title("📝 Phi-3 Mini Article Generator (GPU-powered)")

topic = st.text_input("Enter a topic")

if topic:
    with st.spinner("Generating article..."):
        article = generate_article(topic)
        st.markdown("### Generated Article")
        st.write(article)

In [ ]:
!pkill -f streamlit || echo "No Streamlit process to kill"
from pyngrok import ngrok
ngrok.kill()

NGROK_TOKEN = input("Enter your ngrok authtoken: ").strip()
os.environ["NGROK_AUTHTOKEN"] = NGROK_TOKEN

!ngrok config add-authtoken $NGROK_AUTHTOKEN

In [ ]:
from pyngrok import ngrok
import threading
import os

# Open ngrok tunnel
public_url = ngrok.connect(8501)
print(f"🌐 Streamlit app running at: {public_url}")

# Start Streamlit in background
def run():
    os.system("streamlit run main.py")

thread = threading.Thread(target=run)
thread.start()